In [ ]:
pip install -U imbalanced-learn

In [30]:
pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/b7/f9/9ad1e6e33e4ae149aead1ee8062e536d060b50d650df710899805562910a/streamlit-1.33.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/46/30/2118537233fa72c1d91a81f5908a7e843a6601ccc68b76838ebc4951505f/altair-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/fb/2b/a64c2d25a37aeb921fddb929111413049fc5f8b9a4c1aefaffaafe768d54/cachetools-5.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5,>=3.20 from https://files.pythonhosted.org/packages/ad/6e/1bed3b7c904cc178cb8ee8dbaf72934964452b3de95b7a63412591edb

In [ ]:
#TODO Create Function

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import re # Regular expression library
import string
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

In [21]:
data = pd.read_csv('Hotel Reservations.csv')
df = pd.DataFrame(data)
df = df.drop(columns=['Booking_ID'])
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


In [22]:

# Correcting the 'lower' lambda function
lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) 

# Applying the functions to the DataFrame
# Applying lowercase to all string columns in the DataFrame
for col in df.columns:
    if df[col].dtype == 'object':  # typically string columns are of 'object' type
        df[col] = df[col].map(lower)

encoded_df = df.copy()


encoded_df['type_of_meal_plan'] = encoded_df['type_of_meal_plan'].replace({'not selected': 0, 'meal plan 1': 1, 'meal plan 2': 2, 'meal plan 3': 3})
encoded_df['room_type_reserved'] = encoded_df['room_type_reserved'].replace({'room type 1': 0, 'room type 2': 1, 'room type 3': 2, 'room type 4': 3,
                                                             'room type 5': 4, 'room type 6': 5, 'room type 7': 6})
encoded_df['market_segment_type'] = encoded_df['market_segment_type'].replace({'offline': 0, 'online': 1, 'corporate': 2, 'aviation': 3, 'complementary': 4})
encoded_df['booking_status'] = encoded_df['booking_status'].replace({'canceled': 0, 'not canceled': 1})


In [23]:
X = encoded_df[['no_of_adults', 'no_of_children','no_of_weekend_nights','no_of_week_nights', 'type_of_meal_plan', 	
               'required_car_parking_space', 'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month', 	
               'arrival_date', 'market_segment_type', 'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 	
               'avg_price_per_room', 'no_of_special_requests']]

y = encoded_df['booking_status']

In [24]:
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, y)

In [25]:
#Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Int input 
    no_of_adults
    no_of_children
    no_of_weekend_nights - data picker (if can)
    no_of_week_nights - data picker (if can)
    lead_Time
    Arrival_Year - date picker
    Arrvial_Month - date picker
    Arrival_Date(Days) - date picker
    no_of_pervious_cancellations
    no_of_pervious_booking_not_canceled
    no_of_speical_request

### radio button
    type of meal
    parking needed (Yes Or No)
    room type 
    market_segment_type 
    reqeated_guest (Yes Or No)

### float
    avg_price
    
    
### Type of Meal Plan

    Not Selected = 0
    Meal Plan 1 = 1
    Meal Plan 2 = 2
    Meal Plan 3 = 3

### Room Type

    Room Type 1 = 0
    Room Type 2 = 1
    Room Type 3 = 2
    Room Type 4 = 3
    Room Type 5 = 4
    Room Type 6 = 5
    Room Type 7 = 6

### Market Segment

    offline = 0
    online = 1
    corporate = 2
    avitation = 3
    complementary = 4

### Booking Status 

    canceled = 0
    not canceled = 1
    
    No - 0
    Yes - 1
    
    

### how the system work 

    Step 1 get user data 
    Step 2 validation 
    Step 3 predict
    Step 4 output

In [26]:
data_test = [2,0,0,2,'meal plan 1',0,'Room_Type 1',10,2024,10,20,'Online',0,0,0,65,2]

columns = ["no_of_adults", "no_of_children", "no_of_weekend_nights", "no_of_week_nights",
           "type_of_meal_plan", "required_car_parking_space", "room_type_reserved", "lead_time",
           "arrival_year", "arrival_month", "arrival_date", "market_segment_type",
           "repeated_guest", "no_of_previous_cancellations", "no_of_previous_bookings_not_canceled",
           "avg_price_per_room", "no_of_special_requests"]

df_test = pd.DataFrame([data_test], columns=columns)

lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) 
for col in df_test.columns:
    if df_test[col].dtype == 'object': 
        df_test[col] = df_test[col].map(lower)

df_test['type_of_meal_plan'] = df_test['type_of_meal_plan'].replace({'not selected': 0, 'meal plan 1': 1, 'meal plan 2': 2, 'meal plan 3': 3})
df_test['room_type_reserved'] = df_test['room_type_reserved'].replace({'room type 1': 0, 'room type 2': 1, 'room type 3': 2, 'room type 4': 3,
                                                             'room type 5': 4, 'room type 6': 5, 'room type 7': 6})
df_test['market_segment_type'] = df_test['market_segment_type'].replace({'offline': 0, 'online': 1, 'corporate': 2, 'aviation': 3, 'complementary': 4})

df_test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,2,0,0,2,1,0,0,10,2024,10,20,1,0,0,0,65,2


In [27]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth= 11,random_state=42)

# Train the model
dtc.fit(X_train, y_train)

# Take the model that was trained on the X_train_cv data and apply it to the X_test_cv
prediction = dtc.predict(df_test)

if(prediction == 0):
    str_prediction = 'Canceled'
elif(prediction == 1):
    str_prediction = 'Not Canceled'
print("Prediction result:", str_prediction)

Prediction result: Not Canceled
